In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt


In [ ]:
# Include all packages
import os
import gc
import cv2
import shutil
import numpy as np
import pandas as pd
from time import time
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from yolov5.models.yolo import Model
from sklearn.model_selection import train_test_split

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

import torchvision
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    import zipfile
    with zipfile.ZipFile('/content/drive/MyDrive/DL Project/DataSet.zip', 'r') as zip_ref:
        zip_ref.extractall('./')
except:
    print("Using Local Machine")

In [ ]:
def ResizeImage(image: np.ndarray, x1: int, y1: int, x2: int, y2: int, newWidth: int, newHeight: int) -> tuple:
    originalHeight, originalWidth = image.shape[:2]
    scale = min(newWidth / originalWidth, newHeight / originalHeight)
    resizedImage = cv2.resize(image, (round(originalWidth * scale), round(originalHeight * scale)), interpolation=cv2.INTER_LINEAR)
    dx = round((newWidth - resizedImage.shape[1]) / 2)
    dy = round((newHeight - resizedImage.shape[0]) / 2)
    paddedImage = cv2.copyMakeBorder(resizedImage, dy, dy, dx, dx, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    x1New, y1New = int(x1 * scale + dx), int(y1 * scale + dy)
    x2New, y2New = int(x2 * scale + dx), int(y2 * scale + dy)
    return paddedImage, x1New, y1New, x2New, y2New


In [ ]:
def LoadDataSet(dataSetFolderPath: str) -> tuple:
    images = []
    annotations = []
    annotationsFilePath = dataSetFolderPath+"/allAnnotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=";")
    uniqueSigns = annotationsDataFrame['Annotation tag'].unique().tolist()
    for index, row in annotationsDataFrame[1:].iterrows():
        image = cv2.imread(dataSetFolderPath+"/"+row[0])
        images.append(image)
        annotations.append(
            [row[2], row[3], row[4], row[5]])

    del annotationsDataFrame

    return images, annotations, len(uniqueSigns)


In [ ]:
def PreProcessDataSet(images: list, annotations: list, batchSize: int, resize: tuple) -> tuple:
    resizedImages = []
    newAnnotations = []
    for i, image in enumerate(images):
        [x1, y1, x2, y2] = annotations[i]
        resizedImage, x1New, y1New, x2New, y2New = ResizeImage(
            image, x1, y1, x2, y2, resize[0], resize[1])
        resizedImages.append(resizedImage)
        newAnnotations.append(
            [x1New, y1New, x2New, y2New])

    X_train, X_val, y_train, y_val = train_test_split(
        resizedImages, newAnnotations, test_size=0.3, random_state=42)

    return X_train, X_val, y_train, y_val


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputData, label = self.data[idx]

        if self.transform:
            inputData = self.transform(inputData)
        inputData = torch.from_numpy(inputData).float()
        label = torch.tensor(label).float()
        return inputData, label

def CreateDataLoaders(X_train, X_val, y_train, y_val, batchSize):
    trainDataSet = []
    valDataSet = []
    for i in range(len(X_train)):
        trainDataSet.append((X_train[i], y_train[i]))

    for i in range(len(X_val)):
        valDataSet.append((X_val[i], y_val[i]))

    trainDataSet = CustomDataset(trainDataSet)
    valDataSet = CustomDataset(valDataSet)
    trainDataLoader = DataLoader(
        trainDataSet, batch_size=batchSize, shuffle=True, num_workers=4)
    valDataLoader = DataLoader(
        valDataSet, batch_size=batchSize, shuffle=False, num_workers=4)

    return trainDataLoader, valDataLoader


In [ ]:
def CreateYolov5Model(numClasses: int, version: str):
    congfigFile = "yolov5/models/yolov5{}.yaml".format(version)
    model = Model(congfigFile, ch=3, nc=numClasses)
    # model.load_state_dict(torch.load("yolov5{}.pt".format(version))["model"].state_dict(), strict=False)

    return model


In [ ]:
def DetectImage(model, inputs, device, conf_thres=0.2, iou_thres=0.5):
    model.eval()

    inputs = torch.tensor(inputs, dtype=torch.float32)
    inputs = inputs.unsqueeze(0)
    inputs = inputs.permute(0, 3, 1, 2)
    inputs = inputs.to(device)
    conf_thres = torch.tensor(conf_thres)
    with torch.no_grad():
        output = model(inputs)
        # max_conf_obj_idx = torch.argmax(output[0][..., 4:5], dim=1)
        # output = output[0][torch.arange(output[0].size(0)), max_conf_obj_idx]
        # output = torchvision.ops.nms(output, conf_thres, iou_thres)
        # max_conf_obj_idx = torch.argmax(output[0][..., 4:5], dim=1)
        # output = output[0][torch.arange(output[0].size(0)), max_conf_obj_idx]
        output = output[0]
        box_coordinates = output[..., :4].view(-1, 4)
        confidence_scores = output[..., 4].view(-1)
        nms_indices = torchvision.ops.nms(box_coordinates, confidence_scores, iou_thres)
        output = output.view(-1, output.shape[-1])[nms_indices]
    # Remove the batch dimension
    output = output.squeeze(0)
    return output


In [ ]:
batchSize = 32
inputShape = (640, 640)
epochs = 100
numAnchors = 3
yolo5Version = 'm'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print("Using {} device".format(device))

In [ ]:
images, annotations, numClasses = LoadDataSet("./DataSet")
numClasses = 1


In [ ]:
X_train, X_val, y_train, y_val = PreProcessDataSet(
    images, annotations, batchSize, inputShape)
del images
del annotations
gc.collect()


In [ ]:
# trainDataLoader, valDataLoader = CreateDataLoaders(
#     X_train, X_val, y_train, y_val, batchSize)
# del X_train
# del y_train
# del X_val
# del y_val
gc.collect()


In [ ]:
yolov5Model = CreateYolov5Model(numClasses,yolo5Version)
optimizer = optim.Adam(yolov5Model.parameters(), lr=0.001)
yolov5Model = yolov5Model.to(device)


In [ ]:
savedModels = [ modelName for modelName in os.listdir('/content/drive/MyDrive/DL Project/Trained Models/') if ('yolov5Model' in modelName)]
savedModels.sort()
yolov5Model.load_state_dict(torch.load('/content/drive/MyDrive/DL Project/Trained Models/'+savedModels[-1], map_location=torch.device(device)))


In [ ]:
import random
randInt = random.randint(0,len(X_val))
image = X_val[randInt]
try:
    from google.colab.patches import cv2_imshow
    cv2_imshow(image)
except:
    print("using Local")
    cv2.imshow("Input Image", image)

predictions = DetectImage(yolov5Model, image, device)
[a1,b1,a2,b2] = y_val[randInt]
bBoxs = [[a1,b1,a2,b2]]
i=0
for pred in predictions:
    i+=1
    x1, y1, x2, y2, m1,m2 = pred
    m1,m2, x1, y1, x2, y2= int(m1), int(m2),int(x1), int(y1), int(x2), int(y2)
    if(a1 == x1 or a2 == x2 or b1 == y1 or b2 == y2 ):
      bBoxs.append([x1, y1, x1+x2,y1+ y2])
      
print("No. Objects detected:" ,len(bBoxs) )
crtbBoxs = []
for bBox in bBoxs:
    for x in bBox:
        if(x>=inputShape[0]):
            crtbBoxs.append(bBox)

print("No. Crt Objects detected:" ,len(crtbBoxs) )


In [ ]:
[x1, y1, x2, y2] = bBoxs[0]
cv2.rectangle(image, (x1, y1), (x2, y2), (0,255,0), 2)
for bBox in bBoxs[1:]:
    crtbBox = True
    for x in bBox:
        if(x>=inputShape[0]):
            crtbBox = False
            break

    if(crtbBox):
        [x1, y1, x2, y2] = bBox
        cv2.rectangle(image, (x1, y1), (x2, y2), (0,0,255), 2)
try:
    from google.colab.patches import cv2_imshow
    cv2_imshow(image)
except:
    print("using Local")
    cv2.imshow("Input Image", image)